<a href="https://colab.research.google.com/github/DiwakarBasnet/Neural_Network/blob/main/Sales_Prediction/Sales_function_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Big Mart Sales Prediction
-> Multilayer Perceptron (MLP)

In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# reading dataset
train_data = 'https://raw.githubusercontent.com/DiwakarBasnet/Neural_Network/main/Sales_Prediction/train_file.csv'
dataset1 = pd.read_csv(train_data, skipinitialspace=True)

test_data = 'https://raw.githubusercontent.com/DiwakarBasnet/Neural_Network/main/Sales_Prediction/test_file.csv'
dataset2 = pd.read_csv(test_data, skipinitialspace=True)

##1.Data Preprocessing

In [3]:
dataset1.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
# checking missing values in table
print(dataset1.isnull().sum(),"\n")
print(dataset2.isnull().sum())

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64 

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64


In [5]:
dataset1.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [6]:
def converter(dataset):
  # filling missing values
  dataset['Item_Weight'].fillna(dataset['Item_Weight'].mean(), inplace=True)
  dataset['Outlet_Size'].fillna(dataset['Outlet_Size'].mode()[0], inplace=True)
  
  # converting categorical data to numbers
  dataset['Item_Fat_Content'] = dataset['Item_Fat_Content'].map({'LF':0, 'Low Fat':0, 'low fat':0, 'Regular':1, 'reg':1})
  
  dummy_Item_Type = pd.get_dummies(dataset.Item_Type)
  dummy_Outlet_Establishment_Year = pd.get_dummies(dataset.Outlet_Establishment_Year)
  dummy_Outlet_Size = pd.get_dummies(dataset.Outlet_Size)
  dummy_Outlet_Location_Type = pd.get_dummies(dataset.Outlet_Location_Type)
  dummy_Outlet_Type = pd.get_dummies(dataset.Outlet_Type)

  new_dataset = pd.concat([dataset.Item_Weight, dataset.Item_Fat_Content, dataset.Item_Visibility, dummy_Item_Type, dataset.Item_MRP,
                           dummy_Outlet_Establishment_Year, dummy_Outlet_Size, dummy_Outlet_Location_Type,
                           dummy_Outlet_Type], axis='columns')
  
  # normalizing dataset
  for i in new_dataset.columns[0:]:
    new_dataset[i] = (new_dataset[i] - new_dataset[i].min()) / (new_dataset[i].max() - new_dataset[i].min())

  return new_dataset

In [7]:
# storing all the features as X
X = converter(dataset1)
# storing target as y
y = dataset1['Item_Outlet_Sales']
# shape of X and y
X.shape, y.shape

((8523, 39), (8523,))

In [8]:
##### creating training and validation set #####
# random state to regenerate same train and validation set
# test size 0.2 will keep 20% data in validation and 80% train set

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2)

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6818, 39), (6818,)), ((1705, 39), (1705,)))

##2.Defining the architecture of model

In [9]:
import keras
import tensorflow as tf
from keras import regularizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import InputLayer, Dense
from keras.layers.core import Dropout

In [10]:
# defining input neurons
input_neurons = X_train.shape[1]
# defining output neurons
output_neurons = 1
# defining hidden layers and neurons in each layer
num_of_hidden_layers = 3
neuron_hidden_layer_1 = 30
neuron_hidden_layer_2 = 15
neuron_hidden_layer_3 = 5

In [11]:
from logging import INFO
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu', kernel_initializer='uniform', activity_regularizer=regularizers.l2(0.01)))
model.add(Dropout(rate = 0.2))
model.add(Dense(units=neuron_hidden_layer_2, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate = 0.2))
model.add(Dense(units=neuron_hidden_layer_3, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(rate = 0.2))
model.add(Dense(units=output_neurons, activation='linear', kernel_initializer='uniform'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1200      
                                                                 
 dropout (Dropout)           (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 80        
                                                                 
 dropout_2 (Dropout)         (None, 5)                 0         
                                                                 
 dense_3 (Dense)             (None, 1)                 6

In [13]:
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true))) 

##### Compiling the model #####
model.compile(loss=root_mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

##3.Train model

In [14]:
model_history = model.fit(X_train, y_train, batch_size=128, validation_data=(X_test, y_test), epochs=100)

Epoch 1/100
54/54 [==============================] - 2s 17ms/step - loss: 2755.7627 - mean_squared_error: 7629167.0000 - val_loss: 2794.7007 - val_mean_squared_error: 7831219.5000
Epoch 2/100
54/54 [==============================] - 0s 7ms/step - loss: 2753.4973 - mean_squared_error: 7625387.0000 - val_loss: 2792.1936 - val_mean_squared_error: 7816697.0000
Epoch 3/100
54/54 [==============================] - 0s 8ms/step - loss: 2745.0322 - mean_squared_error: 7559801.0000 - val_loss: 2761.6038 - val_mean_squared_error: 7644611.0000
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: 2669.4683 - mean_squared_error: 7157160.5000 - val_loss: 2623.2087 - val_mean_squared_error: 6895574.5000
Epoch 5/100
54/54 [==============================] - 0s 6ms/step - loss: 2443.3914 - mean_squared_error: 6001123.0000 - val_loss: 2272.8145 - val_mean_squared_error: 5176574.0000
Epoch 6/100
54/54 [==============================] - 0s 8ms/step - loss: 2029.5206 - mean_squared_error:

In [15]:
from sklearn.metrics import r2_score
##### Evaluating model performance on validation set #####
prediction = model.predict(X_test)

# calculating the accuracy on validation set
r2_score(y_test, prediction)

0.5889676148781119

In [16]:
##### Test data preprocessing #####
X_check = converter(dataset2)
X_check.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Baking Goods,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,...,High,Medium,Small,Tier 1,Tier 2,Tier 3,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
0,0.964275,0.0,0.023374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.222983,1.0,0.118737,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.598095,0.0,0.307674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.164335,0.0,0.047548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.484706,1.0,0.366458,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [17]:
pred = model.predict(X_check)

In [18]:
submission = pd.DataFrame()
submission['Item_Identifier'] = dataset2['Item_Identifier']
submission['Outlet_Identifier'] = dataset2['Outlet_Identifier']
submission['Item_Outlet_Sales']= pred

submission

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1518.261108
1,FDW14,OUT017,1235.372803
2,NCN55,OUT010,598.946655
3,FDQ58,OUT017,2369.207275
4,FDY38,OUT027,4798.800781
...,...,...,...
5676,FDB58,OUT046,2051.650879
5677,FDD47,OUT018,2300.256348
5678,NCO17,OUT045,1579.248169
5679,FDJ26,OUT017,3419.037354


In [19]:
submission.to_csv('solution.csv', index=False)